In [1]:
import pathlib
import os
import pickle
import collections
import timeit

import optimum
import optimum.onnxruntime
import onnxruntime
import datasets
import torch
import torch.nn as nn
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import segmentador
import segmentador.optimize
import eval_model

%load_ext autoreload
%autoreload 2

DEV_RUN = True

QUANTIZED_MODELS_DIR = "quantized_models"
pathlib.Path(QUANTIZED_MODELS_DIR).mkdir(exist_ok=True, parents=True)

In [2]:
curated_df = datasets.Dataset.from_file(
    "../data/refined_datasets/df_tokenized_split_0_120000_6000/"
    "combined_test_48_parts_1036_instances/dataset.arrow"
)

curated_df_subsample = curated_df.shard(num_shards=200, index=0)

## Creating LSTM Quantization

In [3]:
segmenter_lstm = segmentador.LSTMSegmenter(
    uri_model="../pretrained_segmenter_model/512_6000_1_lstm/checkpoints/epoch=3-step=3591.ckpt",
    uri_tokenizer="../tokenizers/6000_subwords",
    device="cpu",
)

### With Torch format

In [35]:
quantized_lstm_torch_paths = segmentador.optimize.quantize_model(
    segmenter_lstm,
    model_output_format="torch",
    check_cached=False,
    verbose=True,
)

Saved quantized Pytorch module (Torch format) in './quantized_models/q_num_layers_1_vocab_size_6000_lstm_model.pt'. To use it, load a LSTM segmenter model as:

LSTMSegmenter(
   uri_model='./quantized_models/q_num_layers_1_vocab_size_6000_lstm_model.pt',
   uri_tokenizer='../tokenizers/6000_subwords',
   from_quantized_weights=True,
   ...,
)


In [36]:
segmenter_lstm_torch_quantized = segmentador.LSTMSegmenter(
   uri_model=quantized_lstm_torch_paths.output_uri,
   uri_tokenizer='../tokenizers/6000_subwords',
   from_quantized_weights=True,
)

In [37]:
segmenter_lstm_torch_quantized(curated_df_subsample, show_progress_bar=True)

  0%|          | 0/1 [00:00<?, ?it/s]

['INDICAÇÃO Nº DE 2008 ( Do Sr. Hermes Parcianello )',
 'Indica ao Ministério da Educação a inclusão no ProInfância do município de LARANJEIRAS DO SUL, no estado do Paraná.',
 'Excelentíssimo Senhor Ministro de Estado da Educação, Considerando que a construção de creches e escolas de educação infantil e a reestruturação e aquisição de equipamentos para a rede física escolar desse nível educacional são indispensáveis à melhoria da educação. Considerando um grande percentual de mulheres chefes de família com filhos menores que necessitam de local adequado para deixar seus filhos e que o Programa disponibiliza o atendimento de até 120 crianças em período integral ou até 240 crianças, se distribuídos em turnos matutinos e vespertinos. Considerando que o município ao ser atendido pelo Programa priorizará a acessibilidade, fazendo as adequações necessárias a fim de permitir seu uso por portadores de necessidades especiais. Considerando que a edificação será composta de um bloco administrativ

### With ONNX format

In [43]:
quantized_lstm_onnx_paths = segmentador.optimize.quantize_model(
    segmenter_lstm,
    model_output_format="onnx",
    check_cached=False,
    verbose=True,
)

/media/nvme/segmentador/venvs/env3.9.10/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:2188: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with LSTM can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn("Exporting a model to ONNX with a batch_size other than 1, " +
/media/nvme/segmentador/venvs/env3.9.10/lib/python3.9/site-packages/torch/onnx/symbolic_helper.py:719: UserWarning: allowzero=0 by default. In order to honor zero value in shape use allowzero=1
  warnings.warn("allowzero=0 by default. In order to honor zero value in shape use allowzero=1")
2022-04-09 17:21:26.540632959 [W:onnxruntime:, inference_session.cc:1546 Initialize] Serializing optimized model with Graph Optimization level greater than ORT_ENABLE_EXTENDED and the NchwcTransformer enabled. The generated model ma

Saved quantized Pytorch module (ONNX format) in './quantized_models/q_hidden_layer_dim_512_vocab_size_6000_num_layers_1_opt_level_99_lstm_model.onnx'. To use it, load a LSTM segmenter model as:

segmentador.optimize.quantize.ONNXLSTMSegmenter(
   uri_model='./quantized_models/q_hidden_layer_dim_512_vocab_size_6000_num_layers_1_opt_level_99_lstm_model.onnx',
   uri_tokenizer='../tokenizers/6000_subwords',
   ...,
)


In [44]:
segmenter_lstm_onnx_quantized = segmentador.optimize.ONNXLSTMSegmenter(
   uri_model=quantized_lstm_onnx_paths.output_uri,
   uri_tokenizer='../tokenizers/6000_subwords',
)

In [45]:
segmenter_lstm_onnx_quantized(curated_df_subsample, show_progress_bar=True)

  0%|          | 0/16 [00:00<?, ?it/s]

['INDICAÇÃO Nº DE 2008 ( Do Sr. Hermes Parcianello )',
 'Indica ao Ministério da Educação a inclusão no ProInfância do município de LARANJEIRAS DO SUL, no estado do Paraná.',
 'Excelentíssimo Senhor Ministro de Estado da Educação, Considerando que a construção de creches e escolas de educação infantil e a reestruturação e aquisição de equipamentos para a rede física escolar desse nível educacional são indispensáveis à melhoria da educação. Considerando um grande percentual de mulheres chefes de família com filhos menores que necessitam de local adequado para deixar seus filhos e que o Programa disponibiliza o atendimento de até 120 crianças em período integral ou até 240 crianças, se distribuídos em turnos matutinos e vespertinos. Considerando que o município ao ser atendido pelo Programa priorizará a acessibilidade, fazendo as adequações necessárias a fim de permitir seu uso por portadores de necessidades especiais. Considerando que a edificação será composta de um bloco administrativ

## Creating BERT Quantization

In [10]:
segmenter_bert = segmentador.BERTSegmenter(
    uri_model="../pretrained_segmenter_model/4_6000_layer_model/",
    device="cpu",
)

In [30]:
quantized_bert_paths = segmentador.optimize.quantize_model(
    segmenter_bert,
    check_cached=False,
    verbose=True,
)

/media/nvme/segmentador/venvs/env3.9.10/lib/python3.9/site-packages/torch/onnx/symbolic_helper.py:719: UserWarning: allowzero=0 by default. In order to honor zero value in shape use allowzero=1
  warnings.warn("allowzero=0 by default. In order to honor zero value in shape use allowzero=1")
2022-04-09 17:12:22.543237608 [W:onnxruntime:, inference_session.cc:1546 Initialize] Serializing optimized model with Graph Optimization level greater than ORT_ENABLE_EXTENDED and the NchwcTransformer enabled. The generated model may contain hardware specific optimizations, and should only be used in the same environment the model was optimized in.


failed in shape inference <class 'AssertionError'>
failed in shape inference <class 'AssertionError'>
Saved quantized BERT (ONNX format) in './quantized_models/q_num_layers_4_vocab_size_6000_opt_level_99_bert_model.onnx', and its configuration file in './quantized_models/q_num_layers_4_vocab_size_6000_opt_level_99_bert_model.onnx.config'. To use it, load a BERT segmenter model as:

segmentador.optimize.quantize.ONNXBERTSegmenter(
   uri_model='./quantized_models/q_num_layers_4_vocab_size_6000_opt_level_99_bert_model.onnx',
   uri_tokenizer='../pretrained_segmenter_model/4_6000_layer_model/',
   uri_onnx_config='./quantized_models/q_num_layers_4_vocab_size_6000_opt_level_99_bert_model.onnx.config',
   ...,
)


In [31]:
segmenter_bert_quantized = segmentador.optimize.ONNXBERTSegmenter(
   uri_model=quantized_bert_paths.output_uri,
   uri_tokenizer='../pretrained_segmenter_model/4_6000_layer_model/',
   uri_onnx_config=quantized_bert_paths.onnx_config_uri,
)

In [32]:
segmenter_bert_quantized(curated_df_subsample, show_progress_bar=True)

  0%|          | 0/1 [00:00<?, ?it/s]

['INDICAÇÃO Nº DE 2008 ( Do Sr. Hermes Parcianello )',
 'Indica ao Ministério da Educação a inclusão no ProInfância do município de LARANJEIRAS DO SUL, no estado do Paraná.',
 'Excelentíssimo Senhor Ministro de Estado da Educação, Considerando que a construção de creches e escolas de educação infantil e a reestruturação e aquisição de equipamentos para a rede física escolar desse nível educacional são indispensáveis à melhoria da educação. Considerando um grande percentual de mulheres chefes de família com filhos menores que necessitam de local adequado para deixar seus filhos e que o Programa disponibiliza o atendimento de até 120 crianças em período integral ou até 240 crianças, se distribuídos em turnos matutinos e vespertinos. Considerando que o município ao ser atendido pelo Programa priorizará a acessibilidade, fazendo as adequações necessárias a fim de permitir seu uso por portadores de necessidades especiais. Considerando que a edificação será composta de um bloco administrativ

## Validating performance of distinct configurations

In [14]:
def validate(
    model,
    moving_window_size: int = 1024,
    window_shift_size: float = 0.5,
    batch_size: int = 64,
) -> dict[str, float]:
    sharded_df = curated_df.shard(num_shards=5, index=0)
    
    t_start = timeit.time.perf_counter()
    
    logits = model(
        sharded_df,
        batch_size=batch_size,
        return_logits=True,
        show_progress_bar=True,
        window_shift_size=window_shift_size,
        moving_window_size=moving_window_size,
    ).logits
    
    t_delta = timeit.time.perf_counter() - t_start
    
    metrics = eval_model.compute_metrics(
        ([logits], [np.concatenate(sharded_df["labels"])]),
    )
    metrics["approx_inference_time"] = t_delta
    
    return metrics

In [27]:
validate(segmenter_lstm_onnx_quantized)

  0%|          | 0/4 [00:00<?, ?it/s]

{'per_cls_precision_0': 0.9986046445401736,
 'per_cls_precision_1': 0.9887005649661655,
 'per_cls_precision_2': 0.941605839347328,
 'per_cls_precision_3': 0.6521739127599243,
 'per_cls_recall_0': 0.999543945159263,
 'per_cls_recall_1': 0.9641873278183792,
 'per_cls_recall_2': 0.8164556961508572,
 'per_cls_recall_3': 0.6521739127599243,
 'macro_precision': 0.8952712404033979,
 'macro_recall': 0.858090220472106,
 'macro_f1': 0.8762865031777977,
 'overall_accuracy': 0.9981430927206463,
 'approx_inference_time': 6.539504919997853}

In [46]:
validate(segmenter_lstm_onnx_quantized)

  0%|          | 0/4 [00:00<?, ?it/s]

{'per_cls_precision_0': 0.9986046445401736,
 'per_cls_precision_1': 0.9887005649661655,
 'per_cls_precision_2': 0.941605839347328,
 'per_cls_precision_3': 0.6521739127599243,
 'per_cls_recall_0': 0.999543945159263,
 'per_cls_recall_1': 0.9641873278183792,
 'per_cls_recall_2': 0.8164556961508572,
 'per_cls_recall_3': 0.6521739127599243,
 'macro_precision': 0.8952712404033979,
 'macro_recall': 0.858090220472106,
 'macro_f1': 0.8762865031777977,
 'overall_accuracy': 0.9981430927206463,
 'approx_inference_time': 4.7142820919980295}

In [28]:
validate(segmenter_lstm)

  0%|          | 0/4 [00:00<?, ?it/s]

{'per_cls_precision_0': 0.9986188631961217,
 'per_cls_precision_1': 0.9887069452231017,
 'per_cls_precision_2': 0.941605839347328,
 'per_cls_precision_3': 0.6521739127599243,
 'per_cls_recall_0': 0.999543945159263,
 'per_cls_recall_1': 0.9647382920057039,
 'per_cls_recall_2': 0.8164556961508572,
 'per_cls_recall_3': 0.6521739127599243,
 'macro_precision': 0.895276390131619,
 'macro_recall': 0.8582279615189372,
 'macro_f1': 0.8763607871216822,
 'overall_accuracy': 0.9981569502376564,
 'approx_inference_time': 9.251246863001143}

In [29]:
validate(segmenter_bert)

  0%|          | 0/4 [00:00<?, ?it/s]

{'per_cls_precision_0': 0.9980467636154836,
 'per_cls_precision_1': 0.9197002141278389,
 'per_cls_precision_2': 0.9172932330137373,
 'per_cls_precision_3': 0.5909090906404958,
 'per_cls_recall_0': 0.9976627189418106,
 'per_cls_recall_1': 0.946556473823986,
 'per_cls_recall_2': 0.7721518986853068,
 'per_cls_recall_3': 0.5652173910586011,
 'macro_precision': 0.856487325349389,
 'macro_recall': 0.8203971206274261,
 'macro_f1': 0.8380538480943963,
 'overall_accuracy': 0.9957457422778987,
 'approx_inference_time': 63.15221602300153}

In [34]:
validate(segmenter_bert_quantized)

  0%|          | 0/4 [00:00<?, ?it/s]

{'per_cls_precision_0': 0.998046735767405,
 'per_cls_precision_1': 0.9197431781652234,
 'per_cls_precision_2': 0.9104477611260859,
 'per_cls_precision_3': 0.6190476187528344,
 'per_cls_recall_0': 0.997648467228042,
 'per_cls_recall_1': 0.9471074380113108,
 'per_cls_recall_2': 0.7721518986853068,
 'per_cls_recall_3': 0.5652173910586011,
 'macro_precision': 0.8618213234528872,
 'macro_recall': 0.8205312987458152,
 'macro_f1': 0.8406696150205365,
 'overall_accuracy': 0.9957457422778987,
 'approx_inference_time': 38.16042257900335}